# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [5]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import warnings
import re
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# load data from database
engine = create_engine('sqlite:///Messages.db')
df = pd.read_sql("SELECT * FROM Messages", engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    """
    inputs:
    messages
       
    Returns:
    list of words into numbers of same meaning
    """
    # Converting everything to lower case
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize words
    tokens = word_tokenize(text)
    
    # normalization word tokens and remove stop words
    normlizer = PorterStemmer()
    stop_words = stopwords.words("english")
    
    normlized = [normlizer.stem(word) for word in tokens if word not in stop_words]
    
    return normlized

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
pipe = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1)

np.random.seed(13)
pipe.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [15]:
def eval_metrics(ArrayL, ArrayP, col_names):
    """Evalute metrics of the ML pipeline model
    
    inputs:
    ArrayL: array. Array containing the real labels.
    ArrayP: array. Array containing predicted labels.
    col_names: list of strings. List containing names for each of the ArrayP fields.
       
    Returns:
    data_metrics: Contains accuracy, precision, recall 
    and f1 score for a given set of ArrayL and ArrayP labels.
    """
    metrics = []
    
    # Evaluate metrics for each set of labels
    for i in range(len(col_names)):
        accuracy = accuracy_score(ArrayL[:, i], ArrayP[:, i])
        precision = precision_score(ArrayL[:, i], ArrayP[:, i])
        recall = recall_score(ArrayL[:, i], ArrayP[:, i])
        f1 = f1_score(ArrayL[:, i], ArrayP[:, i])
        
        metrics.append([accuracy, precision, recall, f1])
    
    # store metrics
    metrics = np.array(metrics)
    data_metrics = pd.DataFrame(data = metrics, index = col_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
      
    return data_metrics    

In [19]:
# Calculate evaluation metrics for training set
Y_train_pred = pipe.predict(X_train)
col_names = list(Y.columns.values)

print(eval_metrics(np.array(Y_train), Y_train_pred, col_names))

                        Accuracy  Precision    Recall        F1
related                 0.990267   0.991863  0.995448  0.993652
request                 0.987091   0.996161  0.928444  0.961111
offer                   0.998566   1.000000  0.705263  0.827160
aid_related             0.986374   0.994873  0.972497  0.983558
medical_help            0.988576   0.996302  0.860703  0.923552
medical_products        0.992470   1.000000  0.851815  0.919978
search_and_rescue       0.994416   0.993590  0.814361  0.895091
security                0.995953   0.989051  0.780980  0.872786
military                0.995594   0.994614  0.869702  0.927973
water                   0.994416   1.000000  0.913217  0.954640
food                    0.995390   0.998111  0.960891  0.979147
shelter                 0.992777   0.998748  0.919885  0.957696
clothing                0.997951   0.992647  0.876623  0.931034
money                   0.996568   1.000000  0.855292  0.922002
missing_people          0.997234   1.000

In [20]:
# Calculate evaluation metrics for test set
Y_test_pred = pipe.predict(X_test)

eval_metrics0 = eval_metrics(np.array(Y_test), Y_test_pred, col_names)
print(eval_metrics0)

                        Accuracy  Precision    Recall        F1
related                 0.802367   0.843826  0.909402  0.875388
request                 0.883510   0.790997  0.439286  0.564868
offer                   0.996465   0.000000  0.000000  0.000000
aid_related             0.741048   0.722272  0.602837  0.657172
medical_help            0.919932   0.484848  0.061657  0.109402
medical_products        0.953435   0.673077  0.109034  0.187668
search_and_rescue       0.976794   0.562500  0.058824  0.106509
security                0.980483   0.000000  0.000000  0.000000
military                0.967420   0.703704  0.085202  0.152000
water                   0.944675   0.780000  0.187500  0.302326
food                    0.929461   0.813239  0.475138  0.599826
shelter                 0.931612   0.813084  0.300518  0.438840
clothing                0.985400   0.600000  0.061856  0.112150
money                   0.978638   0.600000  0.042553  0.079470
missing_people          0.987245   0.000

In [21]:
# Calculation the proportion of each column that have label == 1
Y.sum()/len(Y)

related                   0.764792
request                   0.171892
offer                     0.004534
aid_related               0.417243
medical_help              0.080068
medical_products          0.050446
search_and_rescue         0.027816
security                  0.018096
military                  0.033041
water                     0.064239
food                      0.112302
shelter                   0.088904
clothing                  0.015560
money                     0.023206
missing_people            0.011449
refugees                  0.033618
death                     0.045874
other_aid                 0.132396
infrastructure_related    0.065506
transport                 0.046143
buildings                 0.051214
electricity               0.020440
tools                     0.006109
hospitals                 0.010873
shops                     0.004610
aid_centers               0.011872
other_infrastructure      0.044222
weather_related           0.280352
floods              

### 6. Improve your model
Use grid search to find better parameters. 

In [22]:
# Define performance metric for use in grid search scoring object
def perform_metric(Y_T, Y_P):
    """Median F1 score for all classifiers
    
    inputs:
    Y_T: array. Array containing ArrayL labels.
    Y_P: array. Array containing ArrayP labels.
        
    Routputs:
    Median F1 score for all  classifiers
    """
    f1_list = []
    for i in range(np.shape(Y_P)[1]):
        f1 = f1_score(np.array(Y_T)[:, i], Y_P[:, i])
        f1_list.append(f1)
        
    score = np.median(f1_list)
    return score

In [ ]:
# grid search

parameters = {'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__min_samples_split':[2, 5, 10]}

scorer = make_scorer(perform_metric)
cv = GridSearchCV(pipe, param_grid = parameters, scoring = scorer, verbose = 10)

# Find best parameters
np.random.seed(74)
T_model = cv.fit(X_train, Y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.1271186440677966, total=  57.2s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.2min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.11009174311926605, total=  57.2s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.5min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.12195121951219512, total=  57.9s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 


...

[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=1, score=0.11555555555555555, total=  57.8s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  9.5min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=1, score=0.11475409836065573, total=  58.8s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 10.8min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.15339233038348082, total=  47.8s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.15976331360946747, total=  47.4s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.1595744680851064, total=  47.8s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1, score=0.11634349030470914, total= 1.8min
[CV] clf__estimator__min_samples_split=2, cl

In [ ]:
#grid search results
T_model.cv_results_

In [ ]:
# top mean score
np.max(T_model.cv_results_['mean_test_score'])

In [ ]:
# Parameters for top mean score
T_model.best_params_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# evaluating metrics for test set
tuned_pred_test = T_model.predict(X_test)

eval_metrics1 = eval_metrics(np.array(Y_test), tuned_pred_test, col_names)

print(eval_metrics1)

In [ ]:
# summary of 1st model
eval_metrics0.describe()

In [ ]:
# summary of tuned model
eval_metrics1.describe()

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# using SVM instead 
pipe2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(SVC()))
])

parameters2 = {'vect__min_df': [5],
              'tfidf__use_idf':[True],
              'clf__estimator__kernel': ['poly'], 
              'clf__estimator__degree': [1, 2, 3],
              'clf__estimator__C':[1, 10, 100]}

cv2 = GridSearchCV(pipe2, param_grid = parameters2, scoring = scorer, verbose = 10)

# Find best parameters
np.random.seed(77)
T_model2 = cv2.fit(X_train, Y_train)

In [ ]:
# grid search results
T_model2.cv_results_

median always zero therefore lets evaluate metrics

In [ ]:
# Calculate evaluation metrics for test set
tuned_pred_test2 = T_model2.predict(X_test)

eval_metrics2 = eval_metrics(np.array(Y_test), tuned_pred_test2, col_names)

print(eval_metrics2)

SVM doesnt work ,we will be using RANDOM FOREST *FTW*

### 9. Export your model as a pickle file

In [ ]:
# Pickle best model
pickle.dump(T_model, open('disaster_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.